In [4]:
#  1. تثبيت المكتبات
!pip install opencv-python openpyxl pandas tensorflow

#  2. ربط Google Drive
from google.colab import drive
drive.mount('/content/drive')

#  3. تحميل مكتبات بايثون
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from collections import Counter
from google.colab import files

#  4. اختيار ورفع فيديو جديد
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f" تم رفع الفيديو: {video_path}")

#  5. استخراج الإطارات من الفيديو
IMG_SIZE = 128
FRAME_INTERVAL = 10  # كل 10 إطارات

frames = []
cap = cv2.VideoCapture(video_path)
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if count % FRAME_INTERVAL == 0:
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame / 255.0
        frames.append(frame)
    count += 1
cap.release()

frames = np.array(frames)
print(f" عدد الإطارات المستخرجة: {len(frames)}")

#  6. تحميل الموديل الذكي
model = load_model('/content/drive/MyDrive/ScoutPlayResults/smart_shot_full_model.h5')

#  7. تنفيذ التنبؤ
preds = model.predict(frames)
final_preds = [np.argmax(p, axis=1) for p in preds]

#  8. الأغلبية
result_summary = {
    "Shot Power": Counter(final_preds[0]).most_common(1)[0][0],
    "Shot Type": Counter(final_preds[1]).most_common(1)[0][0],
    "Leg Used": Counter(final_preds[2]).most_common(1)[0][0],
    "Shot Technique": Counter(final_preds[3]).most_common(1)[0][0],
    "Shot Result": Counter(final_preds[4]).most_common(1)[0][0],
}

#  9. استرجاع encoders (تم حفظهم مسبقًا أثناء التدريب)
from sklearn.preprocessing import LabelEncoder
encoders = {}

# القيم الأصلية (ضعيها كما كانت أثناء التدريب)
encoders["Shot Power"] = LabelEncoder().fit(["Weak", "Average", "Strong"])
encoders["Shot Type"] = LabelEncoder().fit(["Low Shot", "Direct Shot", "Header", "High Shot"])
encoders["Leg Used"] = LabelEncoder().fit(["Left", "Right"])
encoders["Shot Technique"] = LabelEncoder().fit(["Instep Shot", "Inside Foot", "Knuckleball", "Curl Shot", "Toe Poke", "Half Volley", "Volley"])
encoders["Shot Result"] = LabelEncoder().fit(["Achieved", "Failed"])

#  10. فك ترميز التنبؤ
decoded = {}
for key, encoder in encoders.items():
    decoded[key] = encoder.inverse_transform([result_summary[key]])[0]

print(" النتيجة النهائية للتسديدة:")
for k, v in decoded.items():
    print(f"- {k}: {v}")

#  11. إدخال اسم اللاعب
player_name = input("🧍‍♀️ أدخلي اسم اللاعب: ")

#  12. حفظ النتيجة في Excel
excel_path = "/content/drive/MyDrive/ScoutPlayResults/sorted_players_updated.xlsx"
df_excel = pd.read_excel(excel_path)

new_row = {
    "Name": player_name,
    "Player": "",
    "Shot Power": decoded["Shot Power"],
    "Shot Type": decoded["Shot Type"],
    "Leg Used": decoded["Leg Used"],
    "Shot Technique": decoded["Shot Technique"],
    "Shot Result": decoded["Shot Result"],
    "Video": video_path
}

df_excel = pd.concat([df_excel, pd.DataFrame([new_row])], ignore_index=True)
df_excel.to_excel(excel_path, index=False)

print(" تم حفظ نتيجة التسديدة مع اسم اللاعب في نهاية ملف Excel!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving 1.mp4 to 1.mp4
 تم رفع الفيديو: 1.mp4
 عدد الإطارات المستخرجة: 5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
 النتيجة النهائية للتسديدة:
- Shot Power: Strong
- Shot Type: Low Shot
- Leg Used: Right
- Shot Technique: Inside Foot
- Shot Result: Achieved
🧍‍♀️ أدخلي اسم اللاعب: Omar
 تم حفظ نتيجة التسديدة مع اسم اللاعب في نهاية ملف Excel!
